# Framework for predictions and portfolio forming

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import time

# import the parquet library
import pyarrow.parquet as pq

# import model libraries
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score

In [2]:
# load 'basemodel.parquet'
#df = pd.read_parquet('basemodel.parquet')
df = pd.read_parquet('/kaggle/input/sign-prediction-datasets/basemodel.parquet')
prediction_cols = []
df.head()

,PERMNO,date,RET,ME,bull_D,bear_D,bull_W,bear_W,bull_M,bear_M,LMKT,IVOL,y
0,10000,1986-02-28,-0.257143,11960.00000,0.0,0.0,0.0,1.0,0,0,0.0121,0.004058,0
1,10000,1986-03-31,0.365385,16330.00000,0.0,0.0,0.0,0.0,0,0,0.0766,0.000973,1
2,10000,1986-04-30,-0.098592,15172.00000,0.0,1.0,0.0,0.0,0,0,0.0548,0.001993,0
3,10000,1986-05-31,-0.222656,11793.87834,0.0,0.0,0.0,0.0,0,1,-0.0079,0.000163,0
4,10000,1986-06-30,-0.005025,11734.59375,0.0,0.0,0.0,0.0,0,2,0.0511,0.001569,0


In [3]:
# select the columns to be used for prediction
X_col = ['bull_D', 'bear_D', 'bull_W', 'bear_W', 'bull_M', 'bear_M', 'LMKT', 'IVOL']

In [4]:
# Convert 'date' to datetime format (if not already done) and sort the DataFrame
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='date', inplace=True)
df.reset_index(drop=True, inplace=True)

# Create a 'year' column based on the 'date' column
df['year'] = df['date'].dt.year

# Paper Replication - OLS and Logit, Expanding Window - No Hyperparameters
- They start with out of sample forecasting in 1932
- models will be named model_default

### Linear Regression (Pooled OLS)

In [5]:
#################################
# OLS, default, exp window
#################################

model_name = 'ols_default'


# Update the column name for storing Decision Tree regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

for year in range(df['year'].min() + 6, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Define the training data up until this year
    train_data = df[df['year'] < year]
    
    X_train = train_data[X_col]
    y_train = train_data['y']
    
    # Train the Linear Regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Predict for the next year
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    
    if not X_next_year.empty:
        next_year_predictions = model.predict(X_next_year)
        df.loc[df['year'] == year, model_name] = next_year_predictions
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Time: {iteration_time:.2f} seconds")

Year 1932 - Time: 0.06 seconds
Year 1933 - Time: 0.07 seconds
Year 1934 - Time: 0.05 seconds
Year 1935 - Time: 0.07 seconds
Year 1936 - Time: 0.07 seconds
Year 1937 - Time: 0.07 seconds
Year 1938 - Time: 0.08 seconds
Year 1939 - Time: 0.06 seconds
Year 1940 - Time: 0.06 seconds
Year 1941 - Time: 0.09 seconds
Year 1942 - Time: 0.09 seconds
Year 1943 - Time: 0.10 seconds
Year 1944 - Time: 0.07 seconds
Year 1945 - Time: 0.09 seconds
Year 1946 - Time: 0.11 seconds
Year 1947 - Time: 0.10 seconds
Year 1948 - Time: 0.12 seconds
Year 1949 - Time: 0.13 seconds
Year 1950 - Time: 0.13 seconds
Year 1951 - Time: 0.09 seconds
Year 1952 - Time: 0.10 seconds
Year 1953 - Time: 0.09 seconds
Year 1954 - Time: 0.13 seconds
Year 1955 - Time: 0.17 seconds
Year 1956 - Time: 0.13 seconds
Year 1957 - Time: 0.16 seconds
Year 1958 - Time: 0.11 seconds
Year 1959 - Time: 0.12 seconds
Year 1960 - Time: 0.14 seconds
Year 1961 - Time: 0.16 seconds
Year 1962 - Time: 0.13 seconds
Year 1963 - Time: 0.14 seconds
Year 196

In [6]:
df.tail()

,PERMNO,date,RET,ME,bull_D,bear_D,bull_W,bear_W,bull_M,bear_M,LMKT,IVOL,y,year,ols_default
3627748,14436,2022-07-31,-0.106939,3.731571e+06,2.0,0.0,0.0,0.0,0,0,-0.0837,0.001066,0,2022,0.626592
3627749,14435,2022-07-31,0.020969,5.066263e+03,0.0,0.0,0.0,0.0,0,0,-0.0837,0.003461,1,2022,0.526688
3627750,14434,2022-07-31,0.184785,1.280287e+06,0.0,0.0,3.0,0.0,0,0,-0.0837,0.000616,1,2022,0.799235
3627751,14431,2022-07-31,0.108881,3.475711e+04,0.0,1.0,0.0,0.0,0,0,-0.0837,0.004084,1,2022,0.493767
3627752,93436,2022-07-31,0.323765,9.311106e+08,2.0,0.0,1.0,0.0,0,0,-0.0837,0.000625,1,2022,0.717418


# My Experiments

## Machine Learning - Hyperparameter Tuning included in the process
- models to be named 'model_clas/reg_exp/roll'

### Expanding Window Estimation

In [7]:
#################################
# RIDGE REGRESSION
#################################

model_name = 'ridge_reg_exp'  # Define the model name for Ridge Regression

# Update the column name for storing Ridge regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

# Define the regularization strengths to test
HP1 = [0.01, 0.1, 1, 10, 100]  # Regularization strength alpha for Ridge

for year in range(df['year'].min() + 7, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Define training and tuning datasets
    train_data = df[df['year'] < year]
    tuning_data = train_data[train_data['year'] == year - 1]
    train_data = train_data[train_data['year'] < year - 1]
    
    X_train = train_data[X_col]
    y_train = train_data['y']
    
    X_tune = tuning_data[X_col]
    y_tune = tuning_data['y']
    
    best_HP1 = None
    best_mse = float('inf')  # Initialize with infinity
    
    # Hyperparameter tuning
    for hp1 in HP1:
        model = Ridge(alpha=hp1)  # Ridge regression
        model.fit(X_train, y_train)
        predictions = model.predict(X_tune)  # Predict continuous values
        mse = mean_squared_error(y_tune, predictions)  # Calculate MSE
        
        if mse < best_mse:  # Lower MSE is better
            best_mse = mse
            best_HP1 = hp1
    
    # Retrain on the entire training dataset up to the current year
    X_full_train = df[df['year'] < year][X_col]
    y_full_train = df[df['year'] < year]['y']
    model = Ridge(alpha=best_HP1)  # Ridge regression
    model.fit(X_full_train, y_full_train)
    
    # Predict values for the next year
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    if not X_next_year.empty:
        next_year_predictions = model.predict(X_next_year)  # Predict continuous values
        df.loc[df['year'] == year, model_name] = next_year_predictions
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Best Alpha: {best_HP1}, Best MSE: {round(best_mse,4)}, Time: {iteration_time:.2f} seconds")

Year 1933 - Best Alpha: 100, Best MSE: 0.144, Time: 0.09 seconds
Year 1934 - Best Alpha: 100, Best MSE: 0.1455, Time: 0.09 seconds
Year 1935 - Best Alpha: 0.01, Best MSE: 0.1492, Time: 0.08 seconds
Year 1936 - Best Alpha: 100, Best MSE: 0.1629, Time: 0.09 seconds
Year 1937 - Best Alpha: 100, Best MSE: 0.1561, Time: 0.12 seconds
Year 1938 - Best Alpha: 100, Best MSE: 0.1328, Time: 0.14 seconds
Year 1939 - Best Alpha: 0.01, Best MSE: 0.1651, Time: 0.12 seconds
Year 1940 - Best Alpha: 0.01, Best MSE: 0.1509, Time: 0.12 seconds
Year 1941 - Best Alpha: 0.01, Best MSE: 0.1493, Time: 0.12 seconds
Year 1942 - Best Alpha: 0.01, Best MSE: 0.1475, Time: 0.13 seconds
Year 1943 - Best Alpha: 100, Best MSE: 0.1592, Time: 0.13 seconds
Year 1944 - Best Alpha: 100, Best MSE: 0.1862, Time: 0.13 seconds
Year 1945 - Best Alpha: 0.01, Best MSE: 0.1565, Time: 0.14 seconds
Year 1946 - Best Alpha: 0.01, Best MSE: 0.1383, Time: 0.15 seconds
Year 1947 - Best Alpha: 0.01, Best MSE: 0.1419, Time: 0.15 seconds
Yea

In [8]:
#################################
# DECISION TREE REGRESSION
#################################

model_name = 'DT_reg_exp'  # Define the model name for Decision Tree Regression

# Update the column name for storing Decision Tree regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

# Define the hyperparameters to test - in this case, max_depth of the tree
HP1 = [None, 5, 10, 15, 20, 25]  # Possible values for max_depth

for year in range(df['year'].min() + 7, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Define training and tuning datasets
    train_data = df[df['year'] < year]
    tuning_data = train_data[train_data['year'] == year - 1]
    train_data = train_data[train_data['year'] < year - 1]
    
    X_train = train_data[X_col]
    y_train = train_data['y']
    
    X_tune = tuning_data[X_col]
    y_tune = tuning_data['y']
    
    best_HP1 = None
    best_mse = float('inf')  # Initialize with infinity
    
    # Hyperparameter tuning
    for hp1 in HP1:
        model = DecisionTreeRegressor(max_depth=hp1)  # Decision Tree regression
        model.fit(X_train, y_train)
        predictions = model.predict(X_tune)  # Predict continuous values
        mse = mean_squared_error(y_tune, predictions)  # Calculate MSE
        
        if mse < best_mse:  # Lower MSE is better
            best_mse = mse
            best_HP1 = hp1
    
    # Retrain on the entire training dataset up to the current year
    X_full_train = df[df['year'] < year][X_col]
    y_full_train = df[df['year'] < year]['y']
    model = DecisionTreeRegressor(max_depth=best_HP1)  # Decision Tree regression
    model.fit(X_full_train, y_full_train)
    
    # Predict values for the next year
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    if not X_next_year.empty:
        next_year_predictions = model.predict(X_next_year)  # Predict continuous values
        df.loc[df['year'] == year, model_name] = next_year_predictions
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Best max_depth: {best_HP1}, Best MSE: {round(best_mse,4)}, Time: {iteration_time:.2f} seconds")


Year 1933 - Best max_depth: 10, Best MSE: 0.096, Time: 0.47 seconds
Year 1934 - Best max_depth: 10, Best MSE: 0.0934, Time: 0.52 seconds
Year 1935 - Best max_depth: 10, Best MSE: 0.108, Time: 0.63 seconds
Year 1936 - Best max_depth: 10, Best MSE: 0.1102, Time: 0.73 seconds
Year 1937 - Best max_depth: 10, Best MSE: 0.0987, Time: 0.84 seconds
Year 1938 - Best max_depth: 10, Best MSE: 0.0788, Time: 0.93 seconds
Year 1939 - Best max_depth: 10, Best MSE: 0.085, Time: 1.05 seconds
Year 1940 - Best max_depth: 5, Best MSE: 0.0868, Time: 1.11 seconds
Year 1941 - Best max_depth: 10, Best MSE: 0.1145, Time: 1.27 seconds
Year 1942 - Best max_depth: 10, Best MSE: 0.1177, Time: 1.42 seconds
Year 1943 - Best max_depth: 10, Best MSE: 0.1106, Time: 1.52 seconds
Year 1944 - Best max_depth: 5, Best MSE: 0.0965, Time: 1.58 seconds
Year 1945 - Best max_depth: 10, Best MSE: 0.1117, Time: 1.77 seconds
Year 1946 - Best max_depth: 10, Best MSE: 0.0806, Time: 1.89 seconds
Year 1947 - Best max_depth: 10, Best MS

### First expanding, then rolling
start predicting for 1932, expand the window until you reach X years, then roll it

##### 5 year window

In [9]:
# set the length of the rolling window
rolling_window = 5 # years

In [10]:
#################################
# RIDGE REGRESSION
#################################


model_name = 'ridge_reg_roll5'  # Name for storing Ridge regression predictions

# Predefined set of alpha values for hyperparameter tuning
HP1 = [0.01, 0.1, 1, 10, 100]  # Alpha for Ridge

# Update the column name for storing Ridge regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

# Define the start year for modeling based on having at least 7 years of data
start_modeling_year = df['year'].min() + 7

for year in range(start_modeling_year, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Determine the start year of the training window based on the current year
    train_start_year = max(year - rolling_window, df['year'].min())  # Ensure it does not go below the earliest year
    
    # Select the training data based on the calculated start year
    train_data = df[(df['year'] >= train_start_year) & (df['year'] < year)]
    
    # Split training data into actual training and tuning sets
    # Use the last year of the training data for tuning
    tuning_data = train_data[train_data['year'] == year - 1]
    actual_train_data = train_data[train_data['year'] < year - 1]
    
    X_train = actual_train_data[X_col]
    y_train = actual_train_data['y']
    
    X_tune = tuning_data[X_col]
    y_tune = tuning_data['y']
    
    best_HP1 = None
    best_mse = float('inf')  # Initialize with infinity
    
    # Hyperparameter tuning
    for hp1 in HP1:
        model = Ridge(alpha=hp1)  # Ridge regression
        model.fit(X_train, y_train)
        predictions = model.predict(X_tune)  # Predict continuous values
        mse = mean_squared_error(y_tune, predictions)  # Calculate MSE
        
        if mse < best_mse:  # Lower MSE is better
            best_mse = mse
            best_HP1 = hp1
    
    # Retrain on the entire training window (excluding tuning year) with the best alpha value
    model = Ridge(alpha=best_HP1)
    model.fit(X_train, y_train)
    
    # Predict for the next year
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    
    if not X_next_year.empty:
        next_year_predictions = model.predict(X_next_year)  # Predict continuous values
        df.loc[df['year'] == year, model_name] = next_year_predictions
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Best Alpha: {best_HP1}, Best MSE: {round(best_mse,4)}, Time: {iteration_time:.2f} seconds")

Year 1933 - Best Alpha: 100, Best MSE: 0.1413, Time: 0.07 seconds
Year 1934 - Best Alpha: 100, Best MSE: 0.1459, Time: 0.08 seconds
Year 1935 - Best Alpha: 10, Best MSE: 0.153, Time: 0.07 seconds
Year 1936 - Best Alpha: 100, Best MSE: 0.1844, Time: 0.07 seconds
Year 1937 - Best Alpha: 100, Best MSE: 0.1563, Time: 0.06 seconds
Year 1938 - Best Alpha: 100, Best MSE: 0.1378, Time: 0.07 seconds
Year 1939 - Best Alpha: 0.01, Best MSE: 0.1632, Time: 0.07 seconds
Year 1940 - Best Alpha: 0.01, Best MSE: 0.1493, Time: 0.07 seconds
Year 1941 - Best Alpha: 10, Best MSE: 0.1493, Time: 0.07 seconds
Year 1942 - Best Alpha: 100, Best MSE: 0.1474, Time: 0.07 seconds
Year 1943 - Best Alpha: 100, Best MSE: 0.1643, Time: 0.07 seconds
Year 1944 - Best Alpha: 10, Best MSE: 0.1874, Time: 0.07 seconds
Year 1945 - Best Alpha: 0.01, Best MSE: 0.1579, Time: 0.07 seconds
Year 1946 - Best Alpha: 0.01, Best MSE: 0.1379, Time: 0.07 seconds
Year 1947 - Best Alpha: 1, Best MSE: 0.1577, Time: 0.08 seconds
Year 1948 - 

In [11]:
#################################
# DECISION TREE REGRESSION
#################################

model_name = 'DT_reg_roll5'  # Name for storing Decision Tree regression predictions

# Predefined set of values for hyperparameter tuning (max_depth in this case)
HP1 = [None, 5, 10, 15, 20, 25]  # Possible values for max_depth

# Update the column name for storing Decision Tree regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

# Define the start year for modeling based on having at least 7 years of data
start_modeling_year = df['year'].min() + 7


for year in range(start_modeling_year, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Determine the start year of the training window based on the current year
    train_start_year = max(year - rolling_window, df['year'].min())  # Ensure it does not go below the earliest year
    
    # Select the training data based on the calculated start year
    train_data = df[(df['year'] >= train_start_year) & (df['year'] < year)]
    
    # Split training data into actual training and tuning sets
    # Use the last year of the training data for tuning
    tuning_data = train_data[train_data['year'] == year - 1]
    actual_train_data = train_data[train_data['year'] < year - 1]
    
    X_train = actual_train_data[X_col]
    y_train = actual_train_data['y']
    
    X_tune = tuning_data[X_col]
    y_tune = tuning_data['y']
    
    best_HP1 = None
    best_mse = float('inf')  # Initialize with infinity
    
    # Hyperparameter tuning
    for hp1 in HP1:
        model = DecisionTreeRegressor(max_depth=hp1)  # Decision Tree Regressor
        model.fit(X_train, y_train)
        predictions = model.predict(X_tune)  # Predict continuous values
        mse = mean_squared_error(y_tune, predictions)  # Calculate MSE
        
        if mse < best_mse:  # Lower MSE is better
            best_mse = mse
            best_HP1 = hp1
    
    # Retrain on the entire training window (excluding tuning year) with the best max_depth value
    model = DecisionTreeRegressor(max_depth=best_HP1)
    model.fit(X_train, y_train)
    
    # Predict for the next year
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    
    if not X_next_year.empty:
        next_year_predictions = model.predict(X_next_year)  # Predict continuous values
        df.loc[df['year'] == year, model_name] = next_year_predictions
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Best max_depth: {best_HP1}, Best MSE: {round(best_mse,4)}, Time: {iteration_time:.2f} seconds")


Year 1933 - Best max_depth: 10, Best MSE: 0.0999, Time: 0.34 seconds
Year 1934 - Best max_depth: 10, Best MSE: 0.0859, Time: 0.33 seconds
Year 1935 - Best max_depth: 5, Best MSE: 0.1044, Time: 0.33 seconds
Year 1936 - Best max_depth: 5, Best MSE: 0.1394, Time: 0.35 seconds
Year 1937 - Best max_depth: 10, Best MSE: 0.1047, Time: 0.35 seconds
Year 1938 - Best max_depth: 10, Best MSE: 0.0823, Time: 0.36 seconds
Year 1939 - Best max_depth: 10, Best MSE: 0.0977, Time: 0.36 seconds
Year 1940 - Best max_depth: 5, Best MSE: 0.0988, Time: 0.35 seconds
Year 1941 - Best max_depth: 10, Best MSE: 0.1285, Time: 0.39 seconds
Year 1942 - Best max_depth: 5, Best MSE: 0.1274, Time: 0.39 seconds
Year 1943 - Best max_depth: 5, Best MSE: 0.1321, Time: 0.40 seconds
Year 1944 - Best max_depth: 5, Best MSE: 0.0975, Time: 0.40 seconds
Year 1945 - Best max_depth: 10, Best MSE: 0.1192, Time: 0.40 seconds
Year 1946 - Best max_depth: 5, Best MSE: 0.0856, Time: 0.37 seconds
Year 1947 - Best max_depth: 5, Best MSE: 

##### 10 year window

In [12]:
# set the length of the rolling window
rolling_window = 10 # years

In [13]:
#################################
# RIDGE REGRESSION
#################################


model_name = 'ridge_reg_roll10'  # Name for storing Ridge regression predictions

# Predefined set of alpha values for hyperparameter tuning
HP1 = [0.01, 0.1, 1, 10, 100]  # Alpha for Ridge

# Update the column name for storing Ridge regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

# Define the start year for modeling based on having at least 7 years of data
start_modeling_year = df['year'].min() + 7


for year in range(start_modeling_year, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Determine the start year of the training window based on the current year
    train_start_year = max(year - rolling_window, df['year'].min())  # Ensure it does not go below the earliest year
    
    # Select the training data based on the calculated start year
    train_data = df[(df['year'] >= train_start_year) & (df['year'] < year)]
    
    # Split training data into actual training and tuning sets
    # Use the last year of the training data for tuning
    tuning_data = train_data[train_data['year'] == year - 1]
    actual_train_data = train_data[train_data['year'] < year - 1]
    
    X_train = actual_train_data[X_col]
    y_train = actual_train_data['y']
    
    X_tune = tuning_data[X_col]
    y_tune = tuning_data['y']
    
    best_HP1 = None
    best_mse = float('inf')  # Initialize with infinity
    
    # Hyperparameter tuning
    for hp1 in HP1:
        model = Ridge(alpha=hp1)  # Ridge regression
        model.fit(X_train, y_train)
        predictions = model.predict(X_tune)  # Predict continuous values
        mse = mean_squared_error(y_tune, predictions)  # Calculate MSE
        
        if mse < best_mse:  # Lower MSE is better
            best_mse = mse
            best_HP1 = hp1
    
    # Retrain on the entire training window (excluding tuning year) with the best alpha value
    model = Ridge(alpha=best_HP1)
    model.fit(X_train, y_train)
    
    # Predict for the next year
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    
    if not X_next_year.empty:
        next_year_predictions = model.predict(X_next_year)  # Predict continuous values
        df.loc[df['year'] == year, model_name] = next_year_predictions
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Best Alpha: {best_HP1}, Best MSE: {round(best_mse,4)}, Time: {iteration_time:.2f} seconds")

Year 1933 - Best Alpha: 100, Best MSE: 0.144, Time: 0.07 seconds
Year 1934 - Best Alpha: 100, Best MSE: 0.1455, Time: 0.08 seconds
Year 1935 - Best Alpha: 0.01, Best MSE: 0.1492, Time: 0.08 seconds
Year 1936 - Best Alpha: 100, Best MSE: 0.1629, Time: 0.08 seconds
Year 1937 - Best Alpha: 100, Best MSE: 0.1564, Time: 0.08 seconds
Year 1938 - Best Alpha: 100, Best MSE: 0.1322, Time: 0.07 seconds
Year 1939 - Best Alpha: 0.01, Best MSE: 0.1666, Time: 0.08 seconds
Year 1940 - Best Alpha: 0.01, Best MSE: 0.1507, Time: 0.09 seconds
Year 1941 - Best Alpha: 0.01, Best MSE: 0.149, Time: 0.08 seconds
Year 1942 - Best Alpha: 100, Best MSE: 0.1481, Time: 0.09 seconds
Year 1943 - Best Alpha: 100, Best MSE: 0.1582, Time: 0.09 seconds
Year 1944 - Best Alpha: 100, Best MSE: 0.1769, Time: 0.09 seconds
Year 1945 - Best Alpha: 0.01, Best MSE: 0.1564, Time: 0.09 seconds
Year 1946 - Best Alpha: 0.01, Best MSE: 0.1362, Time: 0.09 seconds
Year 1947 - Best Alpha: 0.01, Best MSE: 0.1433, Time: 0.09 seconds
Year 

In [14]:
#################################
# DECISION TREE REGRESSION
#################################

model_name = 'DT_reg_roll10'  # Name for storing Decision Tree regression predictions

# Predefined set of values for hyperparameter tuning (max_depth in this case)
HP1 = [None, 5, 10, 15, 20, 25]  # Possible values for max_depth

# Update the column name for storing Decision Tree regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

# Define the start year for modeling based on having at least 7 years of data
start_modeling_year = df['year'].min() + 7


for year in range(start_modeling_year, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Determine the start year of the training window based on the current year
    train_start_year = max(year - rolling_window, df['year'].min())  # Ensure it does not go below the earliest year
    
    # Select the training data based on the calculated start year
    train_data = df[(df['year'] >= train_start_year) & (df['year'] < year)]
    
    # Split training data into actual training and tuning sets
    # Use the last year of the training data for tuning
    tuning_data = train_data[train_data['year'] == year - 1]
    actual_train_data = train_data[train_data['year'] < year - 1]
    
    X_train = actual_train_data[X_col]
    y_train = actual_train_data['y']
    
    X_tune = tuning_data[X_col]
    y_tune = tuning_data['y']
    
    best_HP1 = None
    best_mse = float('inf')  # Initialize with infinity
    
    # Hyperparameter tuning
    for hp1 in HP1:
        model = DecisionTreeRegressor(max_depth=hp1)  # Decision Tree Regressor
        model.fit(X_train, y_train)
        predictions = model.predict(X_tune)  # Predict continuous values
        mse = mean_squared_error(y_tune, predictions)  # Calculate MSE
        
        if mse < best_mse:  # Lower MSE is better
            best_mse = mse
            best_HP1 = hp1
    
    # Retrain on the entire training window (excluding tuning year) with the best max_depth value
    model = DecisionTreeRegressor(max_depth=best_HP1)
    model.fit(X_train, y_train)
    
    # Predict for the next year
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    
    if not X_next_year.empty:
        next_year_predictions = model.predict(X_next_year)  # Predict continuous values
        df.loc[df['year'] == year, model_name] = next_year_predictions
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Best max_depth: {best_HP1}, Best MSE: {round(best_mse,4)}, Time: {iteration_time:.2f} seconds")


Year 1933 - Best max_depth: 10, Best MSE: 0.095, Time: 0.44 seconds
Year 1934 - Best max_depth: 10, Best MSE: 0.0928, Time: 0.52 seconds
Year 1935 - Best max_depth: 10, Best MSE: 0.108, Time: 0.61 seconds
Year 1936 - Best max_depth: 10, Best MSE: 0.1102, Time: 0.71 seconds
Year 1937 - Best max_depth: 10, Best MSE: 0.1, Time: 0.80 seconds
Year 1938 - Best max_depth: 5, Best MSE: 0.0806, Time: 0.77 seconds
Year 1939 - Best max_depth: 10, Best MSE: 0.0963, Time: 0.82 seconds
Year 1940 - Best max_depth: 5, Best MSE: 0.087, Time: 0.81 seconds
Year 1941 - Best max_depth: 10, Best MSE: 0.1163, Time: 0.87 seconds
Year 1942 - Best max_depth: 5, Best MSE: 0.1227, Time: 0.84 seconds
Year 1943 - Best max_depth: 5, Best MSE: 0.1165, Time: 0.88 seconds
Year 1944 - Best max_depth: 5, Best MSE: 0.0973, Time: 0.87 seconds
Year 1945 - Best max_depth: 10, Best MSE: 0.1154, Time: 0.93 seconds
Year 1946 - Best max_depth: 10, Best MSE: 0.0817, Time: 0.93 seconds
Year 1947 - Best max_depth: 10, Best MSE: 0.0

##### 20 year window

In [15]:
rolling_window = 20 # years

In [16]:
#################################
# RIDGE REGRESSION
#################################


model_name = 'ridge_reg_roll20'  # Name for storing Ridge regression predictions

# Predefined set of alpha values for hyperparameter tuning
HP1 = [0.01, 0.1, 1, 10, 100]  # Alpha for Ridge

# Update the column name for storing Ridge regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

# Define the start year for modeling based on having at least 7 years of data
start_modeling_year = df['year'].min() + 7


for year in range(start_modeling_year, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Determine the start year of the training window based on the current year
    train_start_year = max(year - rolling_window, df['year'].min())  # Ensure it does not go below the earliest year
    
    # Select the training data based on the calculated start year
    train_data = df[(df['year'] >= train_start_year) & (df['year'] < year)]
    
    # Split training data into actual training and tuning sets
    # Use the last year of the training data for tuning
    tuning_data = train_data[train_data['year'] == year - 1]
    actual_train_data = train_data[train_data['year'] < year - 1]
    
    X_train = actual_train_data[X_col]
    y_train = actual_train_data['y']
    
    X_tune = tuning_data[X_col]
    y_tune = tuning_data['y']
    
    best_HP1 = None
    best_mse = float('inf')  # Initialize with infinity
    
    # Hyperparameter tuning
    for hp1 in HP1:
        model = Ridge(alpha=hp1)  # Ridge regression
        model.fit(X_train, y_train)
        predictions = model.predict(X_tune)  # Predict continuous values
        mse = mean_squared_error(y_tune, predictions)  # Calculate MSE
        
        if mse < best_mse:  # Lower MSE is better
            best_mse = mse
            best_HP1 = hp1
    
    # Retrain on the entire training window (excluding tuning year) with the best alpha value
    model = Ridge(alpha=best_HP1)
    model.fit(X_train, y_train)
    
    # Predict for the next year
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    
    if not X_next_year.empty:
        next_year_predictions = model.predict(X_next_year)  # Predict continuous values
        df.loc[df['year'] == year, model_name] = next_year_predictions
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Best Alpha: {best_HP1}, Best MSE: {round(best_mse,4)}, Time: {iteration_time:.2f} seconds")

Year 1933 - Best Alpha: 100, Best MSE: 0.144, Time: 0.06 seconds
Year 1934 - Best Alpha: 100, Best MSE: 0.1455, Time: 0.07 seconds
Year 1935 - Best Alpha: 0.01, Best MSE: 0.1492, Time: 0.08 seconds
Year 1936 - Best Alpha: 100, Best MSE: 0.1629, Time: 0.09 seconds
Year 1937 - Best Alpha: 100, Best MSE: 0.1561, Time: 0.08 seconds
Year 1938 - Best Alpha: 100, Best MSE: 0.1328, Time: 0.10 seconds
Year 1939 - Best Alpha: 0.01, Best MSE: 0.1651, Time: 0.10 seconds
Year 1940 - Best Alpha: 0.01, Best MSE: 0.1509, Time: 0.11 seconds
Year 1941 - Best Alpha: 0.01, Best MSE: 0.1493, Time: 0.11 seconds
Year 1942 - Best Alpha: 0.01, Best MSE: 0.1475, Time: 0.12 seconds
Year 1943 - Best Alpha: 100, Best MSE: 0.1592, Time: 0.12 seconds
Year 1944 - Best Alpha: 100, Best MSE: 0.1862, Time: 0.12 seconds
Year 1945 - Best Alpha: 0.01, Best MSE: 0.1565, Time: 0.12 seconds
Year 1946 - Best Alpha: 0.01, Best MSE: 0.1383, Time: 0.12 seconds
Year 1947 - Best Alpha: 0.01, Best MSE: 0.1418, Time: 0.13 seconds
Yea

In [17]:
#################################
# DECISION TREE REGRESSION
#################################

model_name = 'DT_reg_roll20'  # Name for storing Decision Tree regression predictions

# Predefined set of values for hyperparameter tuning (max_depth in this case)
HP1 = [None, 5, 10, 15, 20, 25]  # Possible values for max_depth

# Update the column name for storing Decision Tree regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

# Define the start year for modeling based on having at least 7 years of data
start_modeling_year = df['year'].min() + 7


for year in range(start_modeling_year, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Determine the start year of the training window based on the current year
    train_start_year = max(year - rolling_window, df['year'].min())  # Ensure it does not go below the earliest year
    
    # Select the training data based on the calculated start year
    train_data = df[(df['year'] >= train_start_year) & (df['year'] < year)]
    
    # Split training data into actual training and tuning sets
    # Use the last year of the training data for tuning
    tuning_data = train_data[train_data['year'] == year - 1]
    actual_train_data = train_data[train_data['year'] < year - 1]
    
    X_train = actual_train_data[X_col]
    y_train = actual_train_data['y']
    
    X_tune = tuning_data[X_col]
    y_tune = tuning_data['y']
    
    best_HP1 = None
    best_mse = float('inf')  # Initialize with infinity
    
    # Hyperparameter tuning
    for hp1 in HP1:
        model = DecisionTreeRegressor(max_depth=hp1)  # Decision Tree Regressor
        model.fit(X_train, y_train)
        predictions = model.predict(X_tune)  # Predict continuous values
        mse = mean_squared_error(y_tune, predictions)  # Calculate MSE
        
        if mse < best_mse:  # Lower MSE is better
            best_mse = mse
            best_HP1 = hp1
    
    # Retrain on the entire training window (excluding tuning year) with the best max_depth value
    model = DecisionTreeRegressor(max_depth=best_HP1)
    model.fit(X_train, y_train)
    
    # Predict for the next year
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    
    if not X_next_year.empty:
        next_year_predictions = model.predict(X_next_year)  # Predict continuous values
        df.loc[df['year'] == year, model_name] = next_year_predictions
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Best max_depth: {best_HP1}, Best MSE: {round(best_mse,4)}, Time: {iteration_time:.2f} seconds")


Year 1933 - Best max_depth: 10, Best MSE: 0.0955, Time: 0.45 seconds
Year 1934 - Best max_depth: 10, Best MSE: 0.0934, Time: 0.53 seconds
Year 1935 - Best max_depth: 10, Best MSE: 0.108, Time: 0.62 seconds
Year 1936 - Best max_depth: 10, Best MSE: 0.1102, Time: 0.72 seconds
Year 1937 - Best max_depth: 10, Best MSE: 0.0985, Time: 0.84 seconds
Year 1938 - Best max_depth: 10, Best MSE: 0.0788, Time: 0.93 seconds
Year 1939 - Best max_depth: 10, Best MSE: 0.0851, Time: 1.02 seconds
Year 1940 - Best max_depth: 5, Best MSE: 0.0868, Time: 1.11 seconds
Year 1941 - Best max_depth: 10, Best MSE: 0.1145, Time: 1.28 seconds
Year 1942 - Best max_depth: 10, Best MSE: 0.1177, Time: 1.40 seconds
Year 1943 - Best max_depth: 10, Best MSE: 0.1106, Time: 1.58 seconds
Year 1944 - Best max_depth: 5, Best MSE: 0.0965, Time: 1.59 seconds
Year 1945 - Best max_depth: 10, Best MSE: 0.1117, Time: 1.77 seconds
Year 1946 - Best max_depth: 10, Best MSE: 0.0806, Time: 1.89 seconds
Year 1947 - Best max_depth: 10, Best 

## Forming Portfolios, Value-weighted portfolio returns

In [18]:
df.head()

,PERMNO,date,RET,ME,bull_D,bear_D,bull_W,bear_W,bull_M,bear_M,LMKT,IVOL,y,year,ols_default,ridge_reg_exp,DT_reg_exp,ridge_reg_roll5,DT_reg_roll5,ridge_reg_roll10,DT_reg_roll10,ridge_reg_roll20,DT_reg_roll20
0,11148,1926-08-31,0.109924,76287.5,1.0,0.0,4.0,0.0,0,0,0.0318,0.000040,1,1926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10874,1926-08-31,0.168142,16500.0,0.0,0.0,2.0,0.0,2,0,0.0318,0.000537,1,1926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12968,1926-08-31,-0.006464,55683.0,0.0,0.0,0.0,0.0,0,0,0.0318,0.000007,0,1926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12976,1926-08-31,0.156041,41650.0,1.0,0.0,0.0,0.0,0,0,0.0318,0.000070,1,1926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12984,1926-08-31,0.046875,11323.0,1.0,0.0,0.0,0.0,0,0,0.0318,0.000874,1,1926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
prediction_cols
# prediction_cols = ['logit_default','OLS_default','logit_roll6','DT_reg_roll']

['ols_default',
 'ridge_reg_exp',
 'DT_reg_exp',
 'ridge_reg_roll5',
 'DT_reg_roll5',
 'ridge_reg_roll10',
 'DT_reg_roll10',
 'ridge_reg_roll20',
 'DT_reg_roll20']

In [20]:
portfolio = df[['date', 'RET', 'ME', 'y'] + prediction_cols].copy()
portfolio['date'] = pd.to_datetime(portfolio['date'])

# drop rows with missing values
portfolio.dropna(inplace=True)

portfolio.head()

,date,RET,ME,y,ols_default,ridge_reg_exp,DT_reg_exp,ridge_reg_roll5,DT_reg_roll5,ridge_reg_roll10,DT_reg_roll10,ridge_reg_roll20,DT_reg_roll20
49083,1933-01-31,0.036765,31918.875,1,0.700424,0.702497,1.000000,0.700741,1.000000,0.693440,1.000000,0.693440,1.000000
49084,1933-01-31,0.121849,7142.250,1,0.472653,0.477905,0.267922,0.452244,0.340836,0.476971,0.269823,0.476971,0.269823
49085,1933-01-31,0.015625,1560.000,1,0.775651,0.775699,1.000000,0.773611,1.000000,0.755227,1.000000,0.755227,1.000000
49086,1933-01-31,-0.084337,4275.000,0,0.050251,0.060075,0.000000,0.065397,0.000000,0.061732,0.000000,0.061732,0.000000
49087,1933-01-31,-0.037500,447581.750,0,0.369477,0.374155,0.160714,0.357122,0.354545,0.370675,0.160714,0.370675,0.160714


In [21]:
portfolio.tail()

,date,RET,ME,y,ols_default,ridge_reg_exp,DT_reg_exp,ridge_reg_roll5,DT_reg_roll5,ridge_reg_roll10,DT_reg_roll10,ridge_reg_roll20,DT_reg_roll20
3627748,2022-07-31,-0.106939,3.731571e+06,0,0.626592,0.626592,0.771429,0.690926,0.757556,0.699648,0.466019,0.634734,0.449787
3627749,2022-07-31,0.020969,5.066263e+03,1,0.526688,0.526688,0.568068,0.625092,0.757556,0.631219,0.131148,0.567746,0.449787
3627750,2022-07-31,0.184785,1.280287e+06,1,0.799235,0.799235,0.990749,0.856537,0.757556,0.864719,0.987890,0.810660,0.983327
3627751,2022-07-31,0.108881,3.475711e+04,1,0.493767,0.493767,0.568068,0.582893,0.757556,0.591992,0.131148,0.529158,0.449787
3627752,2022-07-31,0.323765,9.311106e+08,1,0.717418,0.717418,0.968750,0.768048,0.757556,0.777450,0.466019,0.715698,0.898927


In [22]:
# Initialize an empty DataFrame to store value-weighted returns for each model
vwreturns = pd.DataFrame(portfolio['date'].unique(), columns=['date'])  # Ensures all dates are included

for pred_col in prediction_cols:
    # Calculate deciles for this prediction
    decile_col = f'decile_{pred_col}'
    portfolio[decile_col] = portfolio.groupby(['date'])[pred_col].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop'))
    
    # Determine position based on deciles
    position_col = f'position_{pred_col}'
    portfolio[position_col] = np.where(portfolio[decile_col] == 9, 1, np.where(portfolio[decile_col] == 0, -1, 0))
    
    # Calculate the value-weighted return for this prediction
    vwret_col = f'vwreturn_{pred_col}'
    vwreturns_temp = portfolio.groupby('date').apply(lambda x: np.sum(x['RET'] * x['ME'] * x[position_col]) / np.sum(x['ME'])).reset_index(name=vwret_col)
    
    # Merge the temporary value-weighted returns with the main vwreturns DataFrame
    vwreturns = vwreturns.merge(vwreturns_temp, on='date', how='left')

# Ensure the 'date' column is the first column and is sorted
vwreturns = vwreturns.sort_values('date').reset_index(drop=True)


/tmp/ipykernel_18/2999037948.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  vwreturns_temp = portfolio.groupby('date').apply(lambda x: np.sum(x['RET'] * x['ME'] * x[position_col]) / np.sum(x['ME'])).reset_index(name=vwret_col)
/tmp/ipykernel_18/2999037948.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  vwreturns_temp = portfolio.groupby('date').apply(lambda x: np.sum(x['RET'] * x['ME'] * x[positi

In [23]:
vwreturns.head()

,date,vwreturn_ols_default,vwreturn_ridge_reg_exp,vwreturn_DT_reg_exp,vwreturn_ridge_reg_roll5,vwreturn_DT_reg_roll5,vwreturn_ridge_reg_roll10,vwreturn_DT_reg_roll10,vwreturn_ridge_reg_roll20,vwreturn_DT_reg_roll20
0,1933-01-31,0.020306,0.020306,0.005217,0.020332,0.006217,0.020332,0.006111,0.020332,0.006111
1,1933-02-28,0.011009,0.011009,0.092540,0.011009,0.065318,0.011009,0.092595,0.011009,0.092595
2,1933-03-31,0.028039,0.028039,0.026435,0.028032,0.026240,0.028039,0.023226,0.028039,0.023226
3,1933-04-30,0.075182,0.075182,-0.018050,0.075182,-0.007120,0.075182,-0.011573,0.075182,-0.011573
4,1933-05-31,0.024181,0.024211,-0.246518,0.024211,-0.246518,0.022286,-0.246518,0.022286,-0.246518


### Compare to market data

In [24]:
#market = pd.read_csv('FF3_clean.csv')
market = pd.read_csv('/kaggle/input/sign-prediction-datasets/FF3_clean.csv')

In [25]:
market.head()

,date,Mkt-RF,SMB,HML,RF
0,1926-07-31,2.96,-2.56,-2.43,0.22
1,1926-08-31,2.64,-1.17,3.82,0.25
2,1926-09-30,0.36,-1.40,0.13,0.23
3,1926-10-31,-3.24,-0.09,0.70,0.32
4,1926-11-30,2.53,-0.10,-0.51,0.31


In [26]:
# create a new 'Mkt' which is a sum of Mkt-RF and RF
market['Mkt'] = market['Mkt-RF'] + market['RF']

# divide all columns by 100 except 'date'
market.iloc[:, 1:] = market.iloc[:, 1:] / 100

#set the 'date' column to datetime format
market['date'] = pd.to_datetime(market['date'])

# merge the market data (only date and Mkt columns) with the vwreturns DataFrame
vwreturns = vwreturns.merge(market[['date', 'Mkt']], on='date', how='left')

# transform all columns (except 'date') to a log: log(x+1) and save the result as lvwreturns
lvwreturns = vwreturns.copy()
lvwreturns.iloc[:, 1:] = np.log(vwreturns.iloc[:, 1:] + 1)

In [27]:
vwreturns.head()

,date,vwreturn_ols_default,vwreturn_ridge_reg_exp,vwreturn_DT_reg_exp,vwreturn_ridge_reg_roll5,vwreturn_DT_reg_roll5,vwreturn_ridge_reg_roll10,vwreturn_DT_reg_roll10,vwreturn_ridge_reg_roll20,vwreturn_DT_reg_roll20,Mkt
0,1933-01-31,0.020306,0.020306,0.005217,0.020332,0.006217,0.020332,0.006111,0.020332,0.006111,0.0126
1,1933-02-28,0.011009,0.011009,0.092540,0.011009,0.065318,0.011009,0.092595,0.011009,0.092595,-0.1527
2,1933-03-31,0.028039,0.028039,0.026435,0.028032,0.026240,0.028039,0.023226,0.028039,0.023226,0.0333
3,1933-04-30,0.075182,0.075182,-0.018050,0.075182,-0.007120,0.075182,-0.011573,0.075182,-0.011573,0.3895
4,1933-05-31,0.024181,0.024211,-0.246518,0.024211,-0.246518,0.022286,-0.246518,0.022286,-0.246518,0.2147


In [28]:
lvwreturns.head()

,date,vwreturn_ols_default,vwreturn_ridge_reg_exp,vwreturn_DT_reg_exp,vwreturn_ridge_reg_roll5,vwreturn_DT_reg_roll5,vwreturn_ridge_reg_roll10,vwreturn_DT_reg_roll10,vwreturn_ridge_reg_roll20,vwreturn_DT_reg_roll20,Mkt
0,1933-01-31,0.020103,0.020103,0.005204,0.020128,0.006198,0.020128,0.006092,0.020128,0.006092,0.012521
1,1933-02-28,0.010949,0.010949,0.088505,0.010949,0.063273,0.010949,0.088556,0.010949,0.088556,-0.165700
2,1933-03-31,0.027653,0.027653,0.026092,0.027646,0.025901,0.027653,0.022960,0.027653,0.022960,0.032758
3,1933-04-30,0.072490,0.072490,-0.018214,0.072490,-0.007145,0.072490,-0.011640,0.072490,-0.011640,0.328944
4,1933-05-31,0.023893,0.023923,-0.283051,0.023923,-0.283051,0.022042,-0.283051,0.022042,-0.283051,0.194497


In [29]:
lvwreturns.describe()

,date,vwreturn_ols_default,vwreturn_ridge_reg_exp,vwreturn_DT_reg_exp,vwreturn_ridge_reg_roll5,vwreturn_DT_reg_roll5,vwreturn_ridge_reg_roll10,vwreturn_DT_reg_roll10,vwreturn_ridge_reg_roll20,vwreturn_DT_reg_roll20,Mkt
count,1074,1074.000000,1074.000000,1074.000000,1074.000000,1074.000000,1074.000000,1074.000000,1074.000000,1074.000000,1074.000000
mean,1977-11-05 12:12:04.022346368,0.015480,0.015481,0.012597,0.015484,0.011327,0.015483,0.012095,0.015494,0.012619,0.008968
min,1933-01-31 00:00:00,-0.019337,-0.019337,-0.283051,-0.019955,-0.283051,-0.019327,-0.283051,-0.019338,-0.283051,-0.272203
25%,1955-06-07 12:00:00,0.010207,0.010210,0.003254,0.010120,0.002764,0.010118,0.002955,0.010195,0.003107,-0.017248
50%,1977-11-15 00:00:00,0.013988,0.013988,0.007472,0.014083,0.007722,0.014084,0.007589,0.014010,0.007395,0.013064
75%,2000-03-23 06:00:00,0.018582,0.018582,0.017071,0.018609,0.018238,0.018619,0.017385,0.018544,0.017291,0.038162
max,2022-07-31 00:00:00,0.086511,0.086511,0.193999,0.085393,0.187124,0.085562,0.193999,0.086511,0.193999,0.328944
std,NaN,0.008332,0.008331,0.020926,0.008246,0.024449,0.008293,0.023904,0.008321,0.021105,0.048186


In [30]:
## plot histograms of the value-weighted returns for each model and the market in lvwreturns
#plt.figure(figsize=(12, round(len(prediction_cols)/2) * 5 ))
#
#for i, pred_col in enumerate(prediction_cols):
#    plt.subplot(len(prediction_cols)/2 +1, 2, i+1)
#    plt.hist(lvwreturns[f'vwreturn_{pred_col}'], bins=50, color='skyblue', edgecolor='black')
#    plt.title(f'Value-Weighted Return - {pred_col}')
#    plt.xlabel('Value-Weighted Return')
#    plt.ylabel('Frequency')
#    # calculate mean, skewness and kurtosis and add their values to the plot as a text, aligning to the top right corner
#    mean = lvwreturns[f'vwreturn_{pred_col}'].mean()
#    skewness = lvwreturns[f'vwreturn_{pred_col}'].skew()
#    kurtosis = lvwreturns[f'vwreturn_{pred_col}'].kurtosis()
#
#    plt.text(0.95, 0.95, f'Mean: {mean:.4f}\nSkewness: {skewness:.4f}\nKurtosis: {kurtosis:.4f}', ha='right', va='top', transform=plt.gca().transAxes)
#
#
#
#plt.subplot(round(len(prediction_cols)/2) +1, 2, len(prediction_cols)+1)
#plt.title('Value-Weighted Return - Market')
#plt.xlabel('Value-Weighted Return')
#plt.ylabel('Frequency')
#plt.hist(lvwreturns['Mkt'], bins=50, color='skyblue', edgecolor='black')
#mean = lvwreturns['Mkt'].mean()
#skewness = lvwreturns['Mkt'].skew()
#kurtosis = lvwreturns['Mkt'].kurtosis()
#plt.text(0.95, 0.95, f'Mean: {mean:.4f}\nSkewness: {skewness:.4f}\nKurtosis: {kurtosis:.4f}', ha='right', va='top', transform=plt.gca().transAxes)
#
#plt.tight_layout()
#plt.show()
#

In [31]:
## plot cumulative sums of the value-weighted log returns
#plt.figure(figsize=(12, 6))
#plt.plot(lvwreturns['date'], lvwreturns.iloc[:, 1:].cumsum())
#plt.title('Cumulative Value-Weighted Log Returns')
#plt.xlabel('Date')
#plt.ylabel('Cumulative Value-Weighted Log Returns')
#plt.legend(prediction_cols + ['Market'])
#plt.show()


In [32]:
# save the lvwreturns and portfolio DataFrame to a parquet file into 'outputs' folder

# for reproducibility and visualization purposes
lvwreturns.to_parquet('base_lvwreturns_reg1.parquet')
portfolio.to_parquet('base_portfolio_reg1.parquet')

# save vwreturns DataFrame to a .dta file into 'outputs' folder
#vwreturns.to_stata('outputs/vwreturns.dta') # for backtasting in R - we need normal returns, not log returns
